In [133]:
!pip install chess
!pip install pandas
!pip install imbalanced-learn
!pip install scikeras[tensorflow]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 80.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.4 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.15.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing inst

In [134]:
!chmod +x /kaggle/input/classification/download_cl.sh
!source /kaggle/input/classification/download_cl.sh

chmod: changing permissions of '/kaggle/input/classification/download_cl.sh': Read-only file system
Files already exist. Skipping download and extraction.


In [135]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as  plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [136]:

def condition(x):
    # Check if the value in column 64 is equal to 1
    return x == 1

def filter_data(x,y,color = 1):
    # Apply the condition to the entire array
    x = x.values
    mask = x[:,64] == color
    filtered_x = x[mask]
    filtered_y = y[mask]
    board = filtered_x[:,0:64].reshape((filtered_x.shape[0],8,8))
    return board,filtered_y

In [137]:
def load_n_datasets(n,foldername):
    all_datasets = []
    filenames = os.listdir(foldername)
    for i in range(n):
        filename = filenames[i]
        if filename.endswith('.csv'):
            dataset = pd.read_csv(os.path.join('classification_data', filename))
            all_datasets.append(dataset)
    combined_dataset = pd.concat(all_datasets, ignore_index=True)
    
    X =  combined_dataset.iloc[:,0:-1]
    y =  combined_dataset["Evaluation"].astype(int)
    
    X,y = filter_data(X,y)
    X = X.reshape(X.shape[0], -1)
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    
    undersampler = RandomUnderSampler()
    X_resampled, y_resampled = undersampler.fit_resample(X, y)
    return X_resampled,y_resampled

In [138]:
# Initialize empty channels for each piece
def bit_map(X):
    channels = np.zeros((X.shape[0],8, 8, 12))  # 12 channels for 6 types of pieces for each player

    # Generate separate channels for each player
    for player in range(2):  # 0 for white pieces, 1 for black pieces
        if player ==0:
            for piece_type in range(6):  # 6 types of pieces
                piece_mask = X == (piece_type + 1)
                channels[:, :, :, player * 6 + piece_type] = piece_mask.astype(np.float32)
        else:
            for piece_type in range(6):  # 6 types of pieces
                piece_mask = -1*(X == (-piece_type - 1))
                channels[:, :, :, player * 6 + piece_type] = piece_mask.astype(np.float32)
    return channels


In [139]:
def one_hot_encoding(y):
    encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoded = encoder.fit_transform(np.array(y).reshape(-1, 1))
    return one_hot_encoded


In [140]:
X_chess_data,y_chess_data = load_n_datasets(2,'classification_data')

X, _, y, _ = train_test_split(X_chess_data, y_chess_data, test_size=0.01, random_state=42, stratify=y_chess_data)

X = np.array(X)
y = np.array(y)
X = X.reshape((X.shape[0],8,8))

[[ 0  0  0 ...  0  0  0]
 [ 0  0 -4 ...  0  0  0]
 [ 0  0  6 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -6  0]
 [ 4  0  0 ...  0  0 -6]
 [ 0  0  0 ...  0  0  0]]


In [141]:
y = one_hot_encoding(y)
X = bit_map(X)

num_classes = len(np.unique(np.argmax(y,axis=1),axis=0))


In [143]:
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val,X_test,y_val,y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)

In [144]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

tf.random.set_seed(101)


In [145]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.activations import elu
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

def create_model(n_neurons=500,n_layers=1,activation="elu",dropout=0.3,l2=0.01):
    model = Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5), activation=elu, input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])))
    model.add(Conv2D(50, kernel_size=(3, 3), activation=elu))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    for i in range(n_layers):
        model.add(Dense(n_neurons, activation=activation,kernel_regularizer=regularizers.l2(l2)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.01), loss="categorical_crossentropy")
    #model.summary()
    return model

In [146]:
def run_model(X_train,y_train,X_val,y_val,X_test,y_test,activation,n_layers,n_neurons,dropout,batch_size,l2,epochs):

    model = create_model(activation=activation,n_layers=n_layers,n_neurons=n_neurons,dropout=dropout,l2=l2)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    # Evaluate the model on the test set
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)

    print("Test Mean Absolute Error:", mae)
    print("Test Root Mean Squared Error:", rmse)
    return history,model

In [149]:
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from scikeras.wrappers import KerasClassifier

def run_random_search_cv(X_data,y_data):
    param_dist = {
        'n_neurons': [100,250,500],
        'n_layers': [1,2],
        'activation':  ['elu','relu','leaky_relu'],
        'dropout':[0,0.3,0.6],
        'l2': [0,0.01,0.05, 0.1],
    }

    keras_model = KerasClassifier(model=create_model, 
                                  n_neurons = param_dist['n_neurons'],
                                  n_layers = param_dist['n_layers'],
                                  activation = param_dist['activation'],
                                  dropout = param_dist['dropout'],
                                  l2 = param_dist['l2'],
                                  verbose=0)
    print(keras_model.get_params().keys())
    random_search = RandomizedSearchCV(estimator=keras_model, 
                                    param_distributions=param_dist, 
                                    n_iter=100, 
                                    scoring = "accuracy",
                                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                    verbose=0)

    random_search.fit(X_data, y_data)

    print(random_search.best_params_)
    print(random_search.best_score_)
    print(random_search.cv_results_)



In [150]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.activations import elu
from tensorflow.keras import regularizers

def create_model2(hp):
    n_neurons = hp.Int('n_neurons', min_value=250, max_value=750, step=250)
    n_layers = hp.Int('n_layers', min_value=1, max_value=2, step=1)
    dropout = hp.Choice('dropout', values=[0.0,0.3,0.6])
    activation = hp.Choice('activation', values=['elu', 'leaky_relu'])
    l2 = hp.Choice('l2', values=[0.0,0.01, 0.1])
    model = Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5), activation=elu, input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])))
    model.add(Conv2D(50, kernel_size=(3, 3), activation=elu))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    for i in range(n_layers):
        model.add(Dense(n_neurons, activation=activation,kernel_regularizer=regularizers.l2(l2)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=['accuracy'])
    #model.summary()
    return model


In [151]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

def custom_evaluation(model, X_val, y_val):
    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(X_val, y_val)
    return accuracy

def run_random_search2(X_train,y_train,X_val, y_val):
    tuner = RandomSearch(
        create_model2,
        objective='val_accuracy',
        max_trials=60,  # Number of hyperparameter combinations to try
        executions_per_trial=1,  # Number of models to build and train for each trial
        directory='test2',  # Directory to save the results
        project_name='test')  # Name for the project


    # Start the hyperparameter search
    tuner.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=128)

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print("The best hyperparameters are:")
    for hparam in best_hps.space:
        name = hparam.name
        value = best_hps.get(name)
        print(f"{name}: {value}")

In [ ]:
grid_search = True
X_data = X
y_data = np.argmax(y,axis=1)
X_val_data = X_val
y_val_data = np.argmax(y_val,axis=1)
num_splits = 5
if not grid_search:
    activation = 'elu'
    n_layers = 1
    n_neurons = 500
    dropout = 0.3
    batch_size = 128
    l2 = 0.01
    epochs = 30
    train_histories = []
    val_histories = []
    models = []
    accuracies = []
    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    for train_index, val_index in skf.split(X_data, y_data):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        history,model = run_model(X_train,y_train,X_val,y_val,X_test,y_test,activation,n_layers,n_neurons,dropout,batch_size,l2,epochs)
        train_histories.append(history.history['loss'])
        val_histories.append(history.history['val_loss'])
        models.append(model)
        y_pred = model.predict(X_test)
        y_pred = np.argmax(y_pred, axis = 1)
        accuracy = np.sum((y_pred==np.argmax(y_test,axis=1))==True)/len(y_pred)
        accuracies.append(accuracy)
        break
else:
    #run_random_search_cv(X_data,y_data)
    print(y_val_data.shape)
    print(y_data.shape)
    run_random_search2(X,y,X_val, y_val)

Trial 5 Complete [00h 01m 36s]
val_accuracy: 0.3271607458591461

Best val_accuracy So Far: 0.3415864109992981
Total elapsed time: 00h 47m 24s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
500               |250               |n_neurons
1                 |1                 |n_layers
0                 |0.3               |dropout
elu               |elu               |activation
0.01              |0                 |l2
256               |128               |batchsize

Epoch 1/20
1891/1891 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.2133 - loss: 2.5220 - val_accuracy: 0.1991 - val_loss: 2.3205
Epoch 2/20
1891/1891 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2611 - loss: 2.0688 - val_accuracy: 0.2623 - val_loss: 2.0500
Epoch 3/20
1891/1891 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2725 - loss: 2.0249 - val_accuracy: 0.2567 - val_loss: 2.0520
Epoch 4/20
1891/1891 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.2794 - loss: 1.9948 - val_accuracy: 0.27

In [ ]:
if not grid_search:
    print("Max accuracy:",np.max(accuracies))
    plt.plot(np.mean(train_histories, axis=0), label='Training Loss')
    plt.plot(np.mean(val_histories, axis=0), label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss over Epochs')
    plt.legend()
    plt.show()